In [1]:
!pip install langchain
!pip install langchain_text_splitters
!pip install -U langchain-community
!pip install pypdf
!pip install nomic

import pandas as pd
import re
from json import loads, dumps
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.embeddings.bedrock import BedrockEmbeddings
# import boto3
from nomic import atlas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.5 MB/s eta 0:00:00
  Created wheel for nomic: filename=nomic-3.3.3-py3-none-any.whl size=49163 sha256=5fb5b8cd95978042ff190041c1f8e98dd4e05f23e141028ca75a50a2c429b6ab
  Stored in directory: /root/.cache/pip/wheels/85/01/6c/a743583e446cb146edb0

In [2]:
from nomic import embed
import numpy as np

!pip install nomic
import nomic
!nomic login
NOMIC_API_KEY = "#Your Pass Nomic Key"
nomic.cli.login(NOMIC_API_KEY)

                                  Authenticate with the Nomic API                                   
                                  https://atlas.nomic.ai/cli-login                                  
           Click the above link to retrieve your access token and then run `nomic login `           


In [3]:
df = pd.read_csv('/content/bbc_news_text_complexity_summarization.csv')

In [6]:
df

,text,labels,no_sentences,Flesch Reading Ease Score,Dale-Chall Readability Score,text_rank_summary,lsa_summary
0,Ad sales boost Time Warner profit\n\nQuarterly...,business,26,62.17,9.72,It hopes to increase subscribers by offering t...,Its profits were buoyed by one-off gains which...
1,Dollar gains on Greenspan speech\n\nThe dollar...,business,17,65.56,9.09,The dollar has hit its highest level against t...,"""I think the chairman's taking a much more san..."
2,Yukos unit buyer faces loan claim\n\nThe owner...,business,14,69.21,9.66,The owners of embattled Russian oil giant Yuko...,Yukos' owner Menatep Group says it will ask Ro...
3,High fuel prices hit BA's profits\n\nBritish A...,business,24,62.98,9.86,Looking ahead to its full year results to Marc...,"Rod Eddington, BA's chief executive, said the ..."
4,Pernod takeover talk lifts Domecq\n\nShares in...,business,17,70.63,10.23,Reports in the Wall Street Journal and the Fin...,Shares in UK drinks and food firm Allied Domec...
...,...,...,...,...,...,...,...
2122,New consoles promise big problems\n\nMaking ga...,tech,52,60.85,9.20,Instead of employing lots of artists to create...,Mr Wright said that enabling players to devise...
2123,BT program to beat dialler scams\n\nBT is intr...,tech,17,56.29,9.23,If a bill rises substantially above its usual ...,BT is introducing two initiatives to help beat...
2124,Be careful how you code\n\nA new European dire...,tech,45,56.29,8.73,"If it gets its way, the Dutch government will ...",A new European directive could put software wr...
2125,US cyber security chief resigns\n\nThe man mak...,tech,16,47.42,9.14,Amit Yoran was director of the National Cyber ...,The man making sure US computer networks are s...


In [7]:
dataset = atlas.map_data(data=df.sample(1000), indexed_field='text')

2024-11-23 02:51:04.281 | WARNING  | nomic.atlas:map_data:128 - An ID field was not specified in your data so one was generated for you in insertion order.
2024-11-23 02:51:04.839 | INFO     | nomic.dataset:_create_project:839 - Organization name: `ebdeskmyhendry`
2024-11-23 02:51:05.267 | INFO     | nomic.dataset:_create_project:867 - Creating dataset `tenacious-poincare`
2024-11-23 02:51:05.394 | INFO     | nomic.atlas:map_data:140 - Uploading data to Atlas.
2024-11-23 02:51:05.426 | WARNING  | nomic.dataset:add_data:1386 - Dropping column __index_level_0__ added in pandas conversion to pyarrow
1it [00:01,  1.02s/it]
2024-11-23 02:51:06.472 | INFO     | nomic.dataset:_add_data:1690 - Upload succeeded.
2024-11-23 02:51:06.474 | INFO     | nomic.atlas:map_data:158 - `ebdeskmyhendry/tenacious-poincare`: Data upload succeeded to dataset`
2024-11-23 02:51:07.299 | INFO     | nomic.dataset:create_index:1272 - Created map `tenacious-poincare` in dataset `ebdeskmyhendry/tenacious-poincare`: 